In [ ]:
ENV["JULIA_PKG_SERVER"] = "https://jp.pkg.julialang.org"

using DifferentialEquations
using Interpolations
using Plots

using LinearAlgebra

using Pkg
using CSV
using DataFrames

using Random

In [ ]:
T0 = 24

GL_tilde(t) = 1.30604 + 0.227902 * sin((2*π/T0)*(t - 3.12764))
TOR_tilde(t) = 1.01103 + 0.0244438 * sin((2*π/T0)*(t - 14.4926))
GI_tilde(t) = 0.423019 + 0.401479 * sin((2*π/T0)*(t - 4.80148))

function NO_light(du,u,p,t)
  # u[1]: TOR
  # u[2]: GI
  # u[3]: R
  # p[1]: GL tilde
  # p[2]: TORt tilde
  # p[3]: GI tilde
    
#     k_GL_TOR = p[4] # init
    k_GL_act = p[4]
    k_GL_phos = p[5] # init
    d_TOR = p[6] # init
    k_phos = p[7] # init
    k_dephos = p[8]
    k_inact = p[9]
    k_act = p[10]
    d_GI = p[11] # init
    g_basal = p[12] # init
    g_TOR = p[13] # init
    K_1 = p[14] # init
    n = p[15]
    sk_TOR = p[16] # init
    K = p[17] # init
    m = p[18]
    d_sk = p[19]
    
    TOR_inact = p[2](t) - u[1]
    GIp = p[3](t) - u[2]

#     du[1] = k_GL_TOR*p[1](t) - k_inact* u[1] * GIp + k_act * TOR_inact - d_TOR * u[1]
    du[1] = k_GL_TOR * p[1](t) * TOR_inact - k_inact* u[1] * GIp - d_TOR * u[1]
    du[2] = - k_phos * u[1] * u[2] + k_dephos * GIp - k_GL_phos * p[1](t) * u[2] - d_GI * u[2]
    du[3] = sk_TOR * u[1]^m / (K + u[1]^m) - d_sk * u[3]
    du[4] = g_basal + g_TOR * u[3]^n / (K_1 + u[3]^n)

end

In [ ]:
u_0 = [0,0,0,200]
# k_GL_TOR = 0.1
k_GL_TOR = 1 # init
k_GL_phos = 0.5 # init
d_TOR = 1 #0.1422 # init
k_phos = 0.5 # init
k_dephos = 3
k_inact = 30
k_act = 1
d_GI = 0.35 # init
g_basal = 50 #32.4016 # init
g_TOR = 70 #64.6040 # init
K_1 = 0.2 # init
n = 2
sk_TOR = 0.5 # init
K = 0.1 # init
m = 2
d_sk = 0.5

tspan = (0,168)
p = (GL_tilde, TOR_tilde, GI_tilde, k_GL_TOR, k_GL_phos, d_TOR, k_phos, k_dephos, k_inact, k_act, d_GI, g_basal, g_TOR, K_1, n, sk_TOR, K, m, d_sk)
prob = ODEProblem(NO_light,u_0,tspan,p)
fit = solve(prob,Vern9(lazy=false); saveat =0.1)

fit_matrix = reduce(hcat, fit.u)

TOR_act = fit_matrix[1,:]
GI = fit_matrix[2,:]
S6K1 = fit_matrix[3,:]
R = fit_matrix[4,:]

TOR_total = TOR_tilde.(fit.t)
GI_total = GI_tilde.(fit.t)

TOR_inact = TOR_total .- TOR_act 
GIp = GI_total .- GI

growth_rate = diff(R) ./ diff(fit.t)
growth_rate = vcat(growth_rate[1], growth_rate)

rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])

tlim = (120,144.2)
tidx = findall(tlim[1] .<= fit.t .<= tlim[2])
xtick = ([tlim[1]:6:tlim[2];],["0","6","12","18","24"])

ylim_GL = (0,maximum(Glucose_data.value)*1.25)
ylim_GI = (0, maximum(GI_data.value) * 1.25)
ylim_TOR = (0, maximum(TOR_data.value) * 1.25)
ylim_S6K1 = (0, maximum(S6K1) * 1.25)
ylim_growth = (minimum(Growth_rate_data.value)*0.8, maximum(Growth_rate_data.value) * 1.25)

lw =5
opacity = 0.3
col = 0.5
color = RGB(col, col, col)
red = RGB(1,0,0)

grid = false
legend = true

P1 = plot(fit.t, GI, linewidth = lw, xlim = tlim, xtick = xtick, ylim = ylim_GI, label = "GI", dpi = 500)
P1 = plot!(fit.t, GI_total, linewidth = lw, color = red, xlim = tlim, xtick = xtick, xaxis = "ZT (hours)", ylim = ylim_GI, dpi = 500)
P1 = plot!(fit.t, GIp, linewidth = lw, xlim = tlim, xtick = xtick, ylim = ylim_GI, label = "GIp", dpi = 500)
P1 = plot!(fontfamily = "arial",legendfontsize = 13, guidefontsize = 19, tickfontsize = 19, xaxis = "ZT (hours)")
P1 = plot!(rectangle(12,2,132,0), opacity = opacity, color = color, linewidth = 0, label = "", dpi = 500)
P1 = plot!(grid = grid, legend = legend)

P2 = plot(fit.t, TOR_act, linewidth = lw, xlim = tlim, xtick = xtick, xaxis = "ZT (hours)", ylim = ylim_TOR, dpi = 500)
P2 = plot!(fit.t, TOR_total, linewidth = lw, color = red, xlim = tlim, xtick = xtick, xaxis = "ZT (hours)", ylim = ylim_TOR, dpi = 500)
P2 = plot!(fit.t, TOR_inact, linewidth = lw, xlim = tlim, xtick = xtick, xaxis = "ZT (hours)", ylim = ylim_TOR, dpi = 500)
P2 = plot!(fontfamily = "arial", legendfontsize = 13, guidefontsize = 19, tickfontsize = 19, xaxis = "ZT (hours)")
P2 = plot!(rectangle(12,2,132,0), opacity = opacity , color = color, linewidth = 0, label = "", dpi = 500)
P2 = plot!(grid = grid, legend = legend)

P3 = plot(fit.t, S6K1, linewidth = lw, xtick = xtick, xlim = tlim, xaxis = "ZT (hours)", ylim = ylim_S6K1, label = "S6K1", dpi = 500)
P3 = plot!(fontfamily = "arial", legendfontsize = 13, guidefontsize = 19, tickfontsize = 19, xaxis = "ZT (hours)")
P3 = plot!(rectangle(12,2,132,0), opacity = opacity, color = color, linewidth = 0, label = "", dpi = 500)
P3 = plot!(grid = grid, legend = legend)

P4 = plot(fit.t, growth_rate, linewidth = lw, xtick = xtick, xlim = tlim, xaxis = "ZT (hours)", ylim = ylim_growth, label = "Growth rate (μm⋅hr^-1)", dpi = 500)
P4 = plot!(fontfamily = "arial", legendfontsize = 13, guidefontsize = 19, tickfontsize = 19, xaxis = "ZT (hours)")
P4 = plot!(Growth_rate_data.time .+ 120, Growth_rate_data.value, seriestype =:scatter, markersize = 7, label = "data")
P4 = plot!(rectangle(12,150,132,0), opacity = opacity, color = color, linewidth = 0, label = "", dpi = 500)
P4 = plot!(grid = grid, legend = legend)

display(P1)
display(P2)
display(P3)
display(P4)
